<a href="https://colab.research.google.com/github/maexx393/hivemindcolab/blob/main/Copy_of_collaborative_training_v0_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://i.imgur.com/FHMoW3N.png" width=360px><br><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Collaborative training <sup>v0.9 alpha</sup></b></center>


This notebook will use local or colab GPU to help train ALBERT-large collaboratively. Your instance will compute gradients and exchange them with a bunch of volunteers around the world. We explain how it works at the bottom. But for now, please run all cells :)

__Warning:__ this is a test run, it will be terminated by __23:59 21 april GMT+0__. By participating, you help us improve the training infrastructure!


_Co-developed by [leshanbog@](https://github.com/leshanbog), [yhn112@](https://github.com/yhn112) and [foksly@](https://github.com/foksly) from YSDA, developers from [hivemind](https://github.com/learning-at-home/hivemind) and [huggingface](http://huggingface.co)_.

In [1]:
!pip install transformers datasets sentencepiece torch_optimizer==0.1.0
!git clone https://github.com/learning-at-home/hivemind -b master
!cd hivemind && pip install -e .
!curl -L https://hivemind-data.s3.us-east-2.amazonaws.com/wikitext103.tar.gz | tar xzf -

import torch
assert torch.cuda.is_available(), "GPU device not found. If running in colab, please retry in a few minutes."
device_name = torch.cuda.get_device_name(0)
if device_name.endswith('T4') or device_name.endswith('P100'):
  microbatch_size = 4
else:
  microbatch_size = 1
print(f"Running with device {device_name}, local batch size = {microbatch_size}")

!ulimit -n 4096 && HIVEMIND_THREADS=256 python ./hivemind/examples/albert/run_trainer.py \
 --client_mode --initial_peers 3.14.12.209:31337 --averaging_expiration 10 --statistics_expiration 120 \
 --batch_size_lead 300 --per_device_train_batch_size {microbatch_size} --gradient_accumulation_steps 1 \
 --logging_first_step --logging_steps 100  --output_dir ./outputs --overwrite_output_dir --logging_dir ./logs \
 --experiment_prefix albert-wikitext-v12 --seed 42

     |████████████████████████████████| 2.8 MB 4.3 MB/s 
     |████████████████████████████████| 264 kB 46.1 MB/s 
     |████████████████████████████████| 1.2 MB 41.4 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 3.3 MB 37.6 MB/s 
     |████████████████████████████████| 636 kB 42.9 MB/s 
     |████████████████████████████████| 119 kB 47.3 MB/s 
     |████████████████████████████████| 243 kB 43.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'hivemind'...
remote: Enumerating objects: 6268, done.
remote: Counting objects: 100% (2875/2875), done.
remote: Compressing objects: 100% (1237/1237), done.
remote: Total 6268 (delta 2006), reused 2310 (delta 1578), pack-reused 3393
Receiving objects: 100% (6268

### What's up next?
* Check the training progress on public learning curves: https://wandb.ai/yhn112/Demo-run/runs/1ygmigce
* See [this tutorial](https://github.com/learning-at-home/hivemind/tree/master/examples/albert) on how to start your own collaborative runs!


### How it works

Since peers can join and leave at any time, we can't use global [Ring All-Reduce](https://towardsdatascience.com/visual-intuition-on-ring-allreduce-for-distributed-deep-learning-d1f34b4911da) for averaging: a single missing peer can break the entire protocol. Instead, peers dynamically assemble into small groups and run all-reduce within each group. Consider an example with 9 GPUs:

<center>
<img src="https://i.imgur.com/QcD1mfG.png" width=360px><br>
The All-Reduce protocol within group can be Ring-AllReduce, but we use a simpler all-to-all algorithm known as butterfly-like all-reduce.<br>
<img src="https://i.imgur.com/ewq3vS6.png" width=380px><br>
After each successful round, participants shuffle around and find new groups:<br>
<img src="https://i.imgur.com/dexNCL3.png" width=350px>

If one of the peers fails to do his part, it will only affect his local group, and only for a single round.


<img src="https://i.imgur.com/RBmElUi.png" width=340px>

Afterwards, peers from the failed group will find new groupmates according to the [moshpit algorithm](https://arxiv.org/abs/2103.03239).

</center>


If you want to learn more and even host your own collaborative experiments, take a look at the [hivemind library](https://github.com/learning-at-home/hivemind/) or the [Moshpit-SGD paper](https://arxiv.org/pdf/2103.03239.pdf).


